# Configuring IPython

## Finding configuration options

IPython has many configurable attributes. These can be viewed using the `-h` flag to the command line applications:

In [1]:
!ipython -h

 IPython

Tools for Interactive Computing in Python

    A Python shell with automatic history (input and output), dynamic object
    introspection, easier configuration, command completion, access to the
    system shell and more.  IPython can also be embedded in running programs.

Usage

    ipython [subcommand] [options] [-c cmd | -m mod | file] [--] [arg] ...

    If invoked with no options, it executes the file and exits, passing the
    remaining arguments to the script, just as if you had specified the same
    command with python. You may need to specify `--` before args to be passed
    to the script, to prevent IPython from attempting to parse them. If you
    specify the option `-i` before the filename, it will enter an interactive
    IPython session after running the script, rather than exiting. Files ending
    in .py will be treated as normal Python, but files ending in .ipy can
    contain special IPython syntax (magic commands, shell expansions, etc.).

    Almost all 

This is an important trick for finding out configuration info:
    
    $> ipython [subcommand] --help-all | grep [-C context] PATTERN

`--help-all` exposes everything configurable in IPython,
there is a good chance you will find what you are looking for.

A common configuration question is:

> how do I disable the "Do you really want to exit" message when quitting with `Ctrl-d`?

Well, logically this has to do with `exit`, so let's look for it:

In [2]:
!ipython --help-all | GREP_COLOR='1;31;46' grep --color exit

    If invoked with no options, it executes the file and exits, passing the
    IPython session after running the script, rather than exiting. Files ending
--confirm-exit
    Set to confirm when you try to exit IPython with an EOF (Control-D
    in Unix, Control-Z/Enter in Windows). By typing 'exit' or 'quit',
    you can force a direct exit without any confirmation.
--no-confirm-exit
    Don't prompt the user when exiting.
--TerminalInteractiveShell.confirm_exit=<CBool>
    Set to confirm when you try to exit IPython with an EOF (Control-D in Unix,
    Control-Z/Enter in Windows). By typing 'exit' or 'quit', you can force a
    direct exit without any confirmation.


Which shows me that I can disable the confirmation for a single IPython session with

    $> ipython --no-confirm-exit
    
or I can set the `TerminalInteractiveShell.confirm_exit=False` in a config file,
to have it be the default behavior.

## Configuration principles

Here are the design principles of the IPython configuration system: 

* Configuration is always done using class attributes
* Classes that have configurable attributes are subclasses of `Configurable`
* Attributes that are configurable are typed traitlets objects (`Bool`, `Unicode`, etc.) that have `config=True`
* In config files, configurable attributes can be set using the format `Class.attr_name=the_value`
* At the command line, configurable attributes can be set using the syntax `--Class.attr_name=the_value`
* At the command line, some attributes have shorthands of the form `--attr-name=value`
* Values set at the command line have higher priority than those set in config files

## The IPython Profile

IPython has a notion of 'profiles' - these are directories that live in your IPYTHONDIR,
which contain configuration and runtime information.

Let's create the default profile

In [3]:
!ipython profile create newprofile

[ProfileCreate] Generating default config file: '/home/takluyver/.ipython/profile_newprofile/ipython_config.py'
[ProfileCreate] Generating default config file: '/home/takluyver/.ipython/profile_newprofile/ipython_kernel_config.py'
[ProfileCreate] Generating default config file: '/home/takluyver/.ipython/profile_newprofile/ipython_console_config.py'
[ProfileCreate] Generating default config file: '/home/takluyver/.ipython/profile_newprofile/ipython_qtconsole_config.py'
[ProfileCreate] Generating default config file: '/home/takluyver/.ipython/profile_newprofile/ipython_notebook_config.py'
[ProfileCreate] Generating default config file: '/home/takluyver/.ipython/profile_newprofile/ipython_nbconvert_config.py'


This creates a profile in your IPYTHONDIR (`ipython locate` is a quick way to see where your IPYTHONDIR is),
and populates it with automatically generated default config files.

In [4]:
!ipython locate profile default
!ipython locate profile newprofile

/home/takluyver/.ipython/profile_default
/home/takluyver/.ipython/profile_newprofile


You can skim

In [5]:
profile = get_ipython().profile_dir.location
profile

'/home/takluyver/.ipython/profile_default'

In [6]:
ls $profile

db/                     ipython_nbconvert_config.py  nbconfig/      startup/
history.sqlite          ipython_notebook_config.py   notebook.json  static/
history.sqlite-journal  ipython_qtconsole_config.py  pid/
ipython_config.py       log/                         security/


Let's peek at our config file

In [7]:
pycat $profile/ipython_config.py

## Startup Files

Startup files are simple Python or IPython scripts
that are run whenever you start IPython.
These are a useful way to do super common imports,
or for building database connections to load on startup of a non-default profile.

We can use a startup file to ensure that our `%tic/toc` magics are always defined,
every time we start IPython.

In [8]:
!ls $profile/startup

imports.py  README


In [9]:
!cat $profile/startup/README

This is the IPython startup directory

.py and .ipy files in this directory will be run *prior* to any code or files specified
via the exec_lines or exec_files configurables whenever you load this profile.

Files will be run in lexicographical order, so you can control the execution order of files
with a prefix, e.g.::

    00-first.py
    50-middle.py
    99-last.ipy


Adding common imports, so we never have to forget them again

In [10]:
%%writefile $profile/startup/simpleimports.py

import sys, os, time, re

Writing /home/takluyver/.ipython/profile_default/startup/simpleimports.py


**Restart the kernel** and then run the following cells immediately to verify these scripts have been executed:

In [11]:
sys

<module 'sys' (built-in)>

## Defining your own magic

As we have seen already, IPython has cell and line magics. You can define your own magics using any Python function and the `register_magic_function` method:

In [12]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

In [13]:
@register_line_magic
def sleep(line):
    """A simple function for sleeping"""
    import time
    t = float(line)
    time.sleep(t)

In [14]:
%sleep 2

In [15]:
%sleep?

### Cell Magic

**Cell magics** take two args:

1. the **line** on the same line of the magic 
2. the **cell** the multiline body of the cell after the first line

In [16]:
@register_cell_magic
def dummy(line, cell):
    """dummy cell magic for displaying the line and cell it is passed"""
    print("line: %r" % line)
    print("cell: %r" % cell)

In [17]:
%%dummy this is the line
this
is the
cell

line: 'this is the line'
cell: 'this\nis the\ncell'


For more details on declaring your own magics in more complex scenarios, [see our docs](http://ipython.org/ipython-doc/stable/interactive/reference.html#defining-your-own-magics).